In [1]:
import math
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [7]:
#import data (10 year time frame)
gold_data = pd.read_csv("dataset/gold_data.csv")
silver_data = pd.read_csv("dataset/silver_data.csv")
gas_data = pd.read_csv("dataset/natural_gas_data.csv")
oil_data = pd.read_csv("dataset/crude_oil_data.csv")
coffee_data = pd.read_csv("dataset/coffee_data.csv")
corn_data = pd.read_csv("dataset/corn_data.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/gold_data.csv'

In [ ]:
gold_data = gold_data.set_index('Date')
silver_data = silver_data.set_index('Date')
gas_data = gas_data.set_index('Date')
oil_data = oil_data.set_index('Date')
coffee_data = coffee_data.set_index('Date')
corn_data = corn_data.set_index('Date')

In [ ]:
gold_data['Commodity']='gold'
silver_data['Commodity']='silver'
gas_data['Commodity']='gas'
oil_data['Commodity']='oil'
coffee_data['Commodity']='coffee'
corn_data['Commodity']='corn'
commodity_data = pd.concat([gold_data, silver_data, gas_data, oil_data, coffee_data, corn_data])
commodity_data.head()

In [ ]:
plt.figure(figsize=(15, 8))
plt.title('Commodity History (10-years)')
plt.plot(gold_data['Close'])
plt.plot(silver_data['Close'])
plt.plot(oil_data['Close'])
plt.plot(gas_data['Close'])
plt.plot(corn_data['Close'])
plt.plot(coffee_data['Close'])
plt.legend(['Gold', 'Silver', 'Oil', 'Gas', 'Corn', 'Coffee' ], loc='lower right')
plt.xlabel('Date')
plt.ylabel('Prices ($)')

In [ ]:
commodity_data.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
for i in range(1, 15):
    commodity_data['t-' + str(i)] = commodity_data['Close'].shift(i)

commodity_data.head()

In [ ]:
print(commodity_data.isnull().sum()) #check how many null values are in out dataset
commodity_data.dropna(inplace=True)
commodity_data.head()

In [ ]:
from sklearn.model_selection import train_test_split
scaler = MinMaxScaler()
commodity_data['Close'] = scaler.fit_transform(commodity_data[['Close']])
commodity_data[['t-1', 't-2', 't-3', 't-4','t-5', 't-6','t-7', 't-8','t-9', 't-10','t-11', 't-12', 't-13', 't-14']] = scaler.fit_transform(commodity_data[['t-1', 't-2', 't-3', 't-4','t-5', 't-6','t-7', 't-8','t-9', 't-10','t-11', 't-12', 't-13', 't-14']])

# Split data into train and test sets
X = commodity_data.drop(['Close', 'Commodity'], axis=1)
y = commodity_data['Close']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Reshape input data
X_train = X_train.values # convert to numpy array
X_train = X_train.reshape((X_train.shape[0], 14, 1)) # reshape to 3D tensor
X_test = X_test.values # convert to numpy array
X_test = X_test.reshape((X_test.shape[0], 14, 1)) # reshape to 3D tensor

In [ ]:
#X_train = tf.expand_dims(X_train, 2)
#X_test = tf.expand_dims(X_test, 2)
print("x_train size and shape: {}".format(X_train.shape))
print("y_train size and shape: {}".format(y_train.shape))
print("x_test size and shape: {}".format(X_test.shape))
print("y_test size and shape: {}".format(y_test.shape))

In [ ]:
model = keras.Sequential()
model.add(layers.LSTM(64, input_shape=(14, 1)))
model.add(layers.Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32)

In [ ]:
# Make predictions on test data
predictions = model.predict(X_test)

X = commodity_data.drop(['Close', 'Commodity'], axis=1)
X = X.values # convert to numpy array
X = X.reshape((X.shape[0], 14, 1)) # reshape to 3D tensor
commodity_data['t-15'] = model.predict(X)
commodity_data.head()

In [ ]:
# cols_to_scale = ['t-1', 't-2', 't-3', 't-4', 't-5', 't-6', 't-7', 't-8', 't-9', 't-10', 't-11', 't-12', 't-13', 't-14']
#commodity_data[cols_to_scale] = scaler.inverse_transform(commodity_data[cols_to_scale])


In [ ]:
# #commodity_data['t-15'] = commodity_data['t-15'].reshape(-1, 1)
# commodity_data['t-15'] = scaler.inverse_transform(commodity_data['t-15'])
commodity_data.head()


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import r2_score
import numpy as np

r2 = r2_score(y_test, predictions)
print(r2)
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)

print(mse)
print(mae)

In [ ]:
# import matplotlib.pyplot as plt

# # Select the commodity data for a single commodity
# commodity_name = 'gold'  # Replace this with the name of the commodity you want to plot
# commodity_data_subset = commodity_data[commodity_data['Commodity'] == commodity_name]

# # Get the original data and predictions for the commodity
# original_data = commodity_data_subset['Close']
# predictions = commodity_data_subset['t-15']

# # Plot the original data and predictions
# plt.figure(figsize=(16,8))
# plt.plot(original_data, label='Original Data')
# plt.plot(predictions, label='Predictions')

# # Add a legend and show the plot
# plt.legend()
# plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Select the commodity data for a single commodity
commodity_name = 'gold'  # Replace this with the name of the commodity you want to plot
commodity_data_subset = commodity_data[commodity_data['Commodity'] == commodity_name]

# Get the original data and predictions for the commodity
original_data = commodity_data_subset['Close']
predictions = commodity_data_subset['t-15']

# Get the index of the 15th day for each prediction
prediction_indices = commodity_data_subset.index[14::15]

# Select only the predictions made on the 15th day
predictions_15th_day = predictions[prediction_indices]

# Plot the original data and predictions
plt.figure(figsize=(16,8))
plt.plot(original_data, label='Original Data')
plt.plot(prediction_indices, predictions_15th_day, label='Predictions')

# Add a legend and show the plot
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Select the commodity data for all commodities
commodity_names = ['gold', 'silver', 'gas', 'oil', 'coffee', 'corn']  # Replace this with the names of the commodities you want to plot
commodity_data_subsets = [commodity_data[commodity_data['Commodity'] == commodity_name] for commodity_name in commodity_names]

# Get the original data and predictions for each commodity
original_data = [commodity_data_subset['Close'] for commodity_data_subset in commodity_data_subsets]
predictions = [commodity_data_subset['t-15'] for commodity_data_subset in commodity_data_subsets]

# Get the index of the 15th day for each prediction
prediction_indices = [commodity_data_subset.index[14::15] for commodity_data_subset in commodity_data_subsets]

# Select only the predictions made on the 15th day
predictions_15th_day = [predictions[i][prediction_indices[i]] for i in range(len(predictions))]

# Plot the original data and predictions for each commodity
plt.figure(figsize=(16,8))
for i in range(len(commodity_names)):
    plt.plot(original_data[i], label=f'{commodity_names[i]} Original Data')
    plt.plot(prediction_indices[i], predictions_15th_day[i], label=f'{commodity_names[i]} Predictions')

# Add a legend and show the plot
plt.legend()
plt.show()